In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras

In [2]:
#loaded the dataset
data_set = pd.read_csv('loan_data.csv')
shuffled_data_set = data_set.sample(frac=1, random_state=42).reset_index(drop=True)
print(shuffled_data_set)


       person_age person_gender person_education  person_income  \
0            32.0          male        Associate        96865.0   
1            24.0          male        Associate        56838.0   
2            22.0        female           Master        37298.0   
3            23.0        female         Bachelor        39944.0   
4            42.0          male      High School        67974.0   
...           ...           ...              ...            ...   
44995        26.0          male      High School        88451.0   
44996        25.0        female      High School        34772.0   
44997        33.0        female        Associate        58317.0   
44998        26.0          male           Master       178602.0   
44999        26.0        female      High School       210894.0   

       person_emp_exp person_home_ownership  loan_amnt        loan_intent  \
0                  10              MORTGAGE     7500.0          EDUCATION   
1                   6                  RE

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import tensorflow as tf
from keras.models import Sequential

# Reload the dataset
dataset = pd.read_csv('loan_data.csv')

# Identify categorical columns
categorical_columns = dataset.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)

# Shuffle the dataset
shuffle_dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training and test sets
train_set = shuffle_dataset[:int(len(shuffle_dataset) * 0.8)]
test_set = shuffle_dataset[int(len(shuffle_dataset) * 0.8):]

# Extract labels
train_labels = train_set['loan_status']
test_labels = test_set['loan_status']

# Define subsets of features
highly_important_features = [
    "person_income",
    "loan_amnt",
    "loan_int_rate",
    "credit_score",
    "cb_person_cred_hist_length",
    "loan_percent_income"
]

# Feature subsets for comparison
feature_subsets = [
    highly_important_features,  # Only highly important features
    train_set.drop(['loan_status'], axis=1).columns.tolist()  # All features
]

# Initialize results list
results = []

# Loop through feature subsets
for i, subset in enumerate(feature_subsets):
    print(f"Training model with feature subset {i + 1}: {subset}")
    
    # Select features
    train_features = train_set[subset]
    test_features = test_set[subset]
    
    # Normalize the features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    test_features = scaler.transform(test_features)
    
    # Create a simple neural network model
    model = Sequential()
    model.add(tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(train_features.shape[1],)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Use model checkpointing
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'best_model_subset_{i + 1}.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
    
    # Train the model
    history = model.fit(
        train_features,
        train_labels,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        callbacks=[checkpoint_callback]
    )
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_features, test_labels)
    predictions = (model.predict(test_features) > 0.5).astype("int32")
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    
    # Append results
    results.append({
        "Feature Subset": f"Subset {i + 1} ({'Highly Important Features' if i == 0 else 'All Features'})",
        "Test Accuracy": f"{test_accuracy * 100:.2f} %",
        "Precision": f"{precision * 100:.2f} %",
        "Recall": f"{recall * 100:.2f} %",
        "F1 Score": f"{f1:.4f}"
    })

# Create a results table
results_table = pd.DataFrame(results)

# Display the results table
print("\nModel Performance Comparison:")
print(results_table)

# Save the results table to a CSV file for further analysis
results_table.to_csv("feature_comparison_results.csv", index=False)

Training model with feature subset 1: ['person_income', 'loan_amnt', 'loan_int_rate', 'credit_score', 'cb_person_cred_hist_length', 'loan_percent_income']


c:\Users\corne\Downloads\AIProject\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4384 - loss: 0.9230

900/900 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4385 - loss: 0.9229 - val_accuracy: 0.7518 - val_loss: 0.5823
Epoch 2/10
894/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7735 - loss: 0.5357

900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7736 - loss: 0.5355 - val_accuracy: 0.8028 - val_loss: 0.4517
Epoch 3/10
891/900 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8126 - loss: 0.4393

...

900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8902 - loss: 0.3023 - val_accuracy: 0.8944 - val_loss: 0.2717
Epoch 4/10
869/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8963 - loss: 0.2672

900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8963 - loss: 0.2671 - val_accuracy: 0.8963 - val_loss: 0.2519
Epoch 5/10
891/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8951 - loss: 0.2499

900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8951 - loss: 0.2499 - val_accuracy: 0.8972 - val_loss: 0.2411
Epoch 6/10
895/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8934 - loss: 0.2460

900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8934 - loss: 0.2460 - val_accuracy: 0.8975 - val_loss: 0.2350
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8953 - loss: 0.2384 - val_accuracy: 0.8971 - val_loss: 0.2312
Epoch 8/10
899/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8974 - loss: 0.2313

900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8974 - loss: 0.2313 - val_accuracy: 0.8985 - val_loss: 0.2288
Epoch 9/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8989 - loss: 0.2286 - val_accuracy: 0.8976 - val_loss: 0.2274
Epoch 10/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8957 - loss: 0.2299 - val_accuracy: 0.8978 - val_loss: 0.2259
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8982 - loss: 0.2226
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Model Performance Comparison:
                         Feature Subset Test Accuracy Precision   Recall  \
0  Subset 1 (Highly Important Features)       82.54 %   68.04 %  39.78 %   
1               Subset 2 (All Features)       89.76 %   78.18 %  74.49 %   

  F1 Score  
0   0.5021  
1   0.7629  
